# Environment Prep



In [21]:
import os

## DATA

you only need to move the data once - it will be there between restarts

### tfrecords
the tutorial says you need 2 new files under the training_demo/annotations folder, named 
- test.record
- train.record

In [1]:
import os

In [2]:
!pwd

/home/ec2-user/SageMaker/TF2x_ObjDetect_Tutorial


In [3]:
!ls /home/ec2-user/SageMaker/tensorflow/workspace/training_demo/annotations

train.record-00000-of-00010  val.record-00000-of-00010
train.record-00001-of-00010  val.record-00001-of-00010


## Model
download the ResNet50 640x640 model  

copy the pipeline.config file to my* folder.   Then edit locally with gedit.

In [4]:
# you should see checkpoint, pipeline.config, saved_model
!ls /home/ec2-user/SageMaker/tensorflow/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/

checkpoint  pipeline.config  saved_model


In [6]:
!ls /home/ec2-user/SageMaker/tensorflow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn -l

total 126076
-rw-rw-r-- 1 ec2-user ec2-user       166 Dec 18 20:09 checkpoint
-rw-rw-r-- 1 ec2-user ec2-user 129031837 Dec 18 20:09 ckpt-1.data-00000-of-00001
-rw-rw-r-- 1 ec2-user ec2-user     31398 Dec 18 20:09 ckpt-1.index
-rw-rw-r-- 1 ec2-user ec2-user      4589 Dec 18 19:06 local_mobilenet_v1_ssd_security_scratch_v9_00.config
-rw-r----- 1 ec2-user ec2-user      4834 Dec 18 20:08 pipeline.config
-rw-rw-r-- 1 ec2-user ec2-user      2582 Dec 18 19:39 security41_label_map.pbtxt
-rw-rw-r-- 1 ec2-user ec2-user      5057 Dec 18 19:14 security_label_map.pbtxt
drwxr-xr-x 2 ec2-user ec2-user      4096 Dec 18 20:08 train


## TF 2.x GPU

In [7]:
import tensorflow as tf

In [8]:
print (tf.__version__)

2.1.0


In [9]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [10]:
physical_devices= tf.config.experimental.list_physical_devices('GPU')
print(len(physical_devices))

1


In [11]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1695416394679242498
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8563922678974639616
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 8278455864135837570
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15791672525
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3007098938038362901
physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:1e.0, compute capability: 7.0"
]


In [12]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

tf.Tensor(31.197632, shape=(), dtype=float32)


## Tutorial - Object Detection API Setup

In [19]:
!pip install pycocotools

  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp36-cp36m-linux_x86_64.whl size=102734 sha256=7d4918d67f0805ce7096739cdfaebef2c6aff63949ca67dac3e22ebfdba15400
  Stored in directory: /home/ec2-user/.cache/pip/wheels/d8/c2/ba/8f5306f921c2e79ad7b09effdfed6bd966cfcf8c6fe55422d6
Successfully built pycocotools
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p36/bin/python -m pip install --upgrade pip' command.


In [20]:
import pycocotools

### API

In [26]:
original_cwd = cwd = os.getcwd()
print ("Original working directory:", original_cwd)

Original working directory: /home/ec2-user/SageMaker/TF2x_ObjDetect_Tutorial


In [27]:
os.chdir('/home/ec2-user/SageMaker/tensorflow/models/research/')
cwd = os.getcwd() 
print("Current working directory is:", cwd)

Current working directory is: /home/ec2-user/SageMaker/tensorflow/models/research


In [24]:
# make sure your kernel is correct:  conda_tensorflow2_p36
!python -m pip install .

Processing /home/ec2-user/SageMaker/tensorflow/models/research
     |████████████████████████████████| 8.9 MB 21.8 MB/s eta 0:00:01     |███▊                            | 1.0 MB 21.8 MB/s eta 0:00:01
     |████████████████████████████████| 352 kB 60.2 MB/s eta 0:00:01
     |████████████████████████████████| 840 kB 63.9 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 64.5 MB/s eta 0:00:01
     |████████████████████████████████| 41 kB 1.3 MB/s  eta 0:00:01
     |████████████████████████████████| 2.0 MB 61.3 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 7.6 MB/s  eta 0:00:01
     |████████████████████████████████| 63.8 MB 70.6 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 61.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 55.9 MB/s eta 0:00:01
     |████████████████████████████████| 89 kB 14.6 MB/s eta 0:00:01
     |████████████████████████████████| 56 k

In [28]:
!python object_detection/builders/model_builder_tf2_test.py

Running tests under Python 3.6.10: /home/ec2-user/anaconda3/envs/tensorflow2_p36/bin/python
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model): 4.18s
I1218 21:08:53.999401 139809678935872 test_util.py:2076] time(__main__.ModelBuilderTF2Test.test_create_center_net_model): 4.18s
[       OK ] ModelBuilderTF2Test.test_create_center_net_model
[ RUN      ] ModelBuilderTF2Test.test_create_experimental_model
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_experimental_model): 0.0s
I1218 21:08:54.001228 139809678935872 test_util.py:2076] time(__main__.ModelBuilderTF2Test.test_create_experimental_model): 0.0s
[       OK ] ModelBuilderTF2Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF2Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)): 0.06s
I1218 21:

In [29]:
os.chdir(original_cwd)
cwd = os.getcwd() 
print("back to original working directory:", cwd)

back to original working directory: /home/ec2-user/SageMaker/TF2x_ObjDetect_Tutorial


## train

In [30]:
!cp /home/ec2-user/SageMaker/tensorflow/models/research/object_detection/model_main_tf2.py .

In [31]:
os.chdir('/home/ec2-user/SageMaker/tensorflow/workspace/training_demo/')
cwd = os.getcwd() 
print("Current working directory is:", cwd)

Current working directory is: /home/ec2-user/SageMaker/tensorflow/workspace/training_demo


In [ ]:
!python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config

W1218 21:10:02.218513 140380072408896 cross_device_ops.py:1321] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I1218 21:10:02.218842 140380072408896 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I1218 21:10:02.227448 140380072408896 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1218 21:10:02.227582 140380072408896 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W1218 21:10:02.298274 140380072408896 deprecation.py:339] From /home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/object_detection/model_lib_v2.py:523: StrategyBase.experimental_distribute_datasets_from_function (